In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#Now we define a new dataframe called data from yfinance which will contain the stock price of apple on a specified date range.
# Define the stock ticker and date range
ticker = 'NVDA'
start_date = '2010-04-01'
end_date = '2024-11-23'

# Fetch the data using yfinance
data = yf.download(ticker, start=start_date, end=end_date)

# Display the DataFrame
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-04-01,0.437000,0.442000,0.426000,0.430500,0.394832,339664000
2010-04-05,0.433250,0.437750,0.432750,0.437000,0.400794,413636000
2010-04-06,0.431500,0.432500,0.418750,0.426250,0.390935,809260000
2010-04-07,0.423500,0.436250,0.420000,0.429000,0.393457,817468000
2010-04-08,0.426250,0.429500,0.419750,0.422000,0.387037,1037600000
...,...,...,...,...,...,...
2024-11-18,139.500000,141.550003,137.149994,140.149994,140.149994,221866000
2024-11-19,141.320007,147.130005,140.990005,147.009995,147.009995,227834900
2024-11-20,147.410004,147.559998,142.729996,145.889999,145.889999,309871700


In [3]:
import numpy as np
data['VWAP'] = (data['Close'] * data['Volume']).cumsum() / data['Volume'].cumsum()

# Calculate the log difference for 'Open', 'Close', 'High', 'Low', 'Volume'
columns_to_calculate = ['Open', 'Close', 'High', 'Low', 'Volume', 'VWAP']
for column in columns_to_calculate:
    data[f'log_diff_{column}'] = np.log(data[column] / data[column].shift(1))

data.loc[:, 'Target'] = (data['Close'].shift(-1) > data['Close']).astype(int)


# List of columns to calculate logarithms for
columns_to_log = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'VWAP']

# Ensure the columns exist in the DataFrame before processing
existing_columns = [col for col in columns_to_log if col in data.columns]

# Add new columns for logarithms
for column in existing_columns:
    data[f'log_{column}'] = np.log(data[column])

In [4]:
horizons = [2,5,60,250,1000]
new_predictors = []
data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
for horizon in horizons:
    rolling_averages = data.rolling(horizon).mean()
    
    ratio_column = f"Close_Ratio_{horizon}"
    data[ratio_column] = data["Close"] / rolling_averages["Close"]
    
    trend_column = f"Trend_{horizon}"
    data[trend_column] = data.shift(1).rolling(horizon).sum()["Target"]
    data[f'Volatility_{horizon}'] = data['Log_Returns'].rolling(window=horizon).std()

    new_predictors+= [ratio_column, trend_column]

# Ensure 'Close' contains the daily closing prices


# Calculate rolling volatility (e.g., 20-day rolling window)
# Remove rows with any null values
data = data.dropna()
data

,Open,High,Low,Close,Adj Close,Volume,VWAP,log_diff_Open,log_diff_Close,log_diff_High,...,Volatility_5,Close_Ratio_60,Trend_60,Volatility_60,Close_Ratio_250,Trend_250,Volatility_250,Close_Ratio_1000,Trend_1000,Volatility_1000
Date,,,,,,,,,,,,,,,,,,,,,
2014-03-24,0.463750,0.467500,0.454500,0.461250,0.437031,306096000,0.374334,-0.009657,-0.004866,-0.009580,...,0.012553,1.088292,31.0,0.015362,1.211940,127.0,0.014113,1.270916,489.0,0.025221
2014-03-25,0.466250,0.466750,0.457750,0.461250,0.437031,304424000,0.374377,0.005376,0.000000,-0.001606,...,0.008701,1.085326,31.0,0.015284,1.210096,126.0,0.014098,1.270831,488.0,0.025216
2014-03-26,0.465500,0.474250,0.446750,0.450750,0.427082,557756000,0.374447,-0.001610,-0.023027,0.015941,...,0.009861,1.058264,30.0,0.015634,1.180938,125.0,0.014160,1.241818,488.0,0.025214
2014-03-27,0.449750,0.453500,0.439750,0.444750,0.421397,540728000,0.374509,-0.034420,-0.013400,-0.044739,...,0.009590,1.042322,29.0,0.015695,1.163579,125.0,0.014019,1.225234,487.0,0.025217
2014-03-28,0.444000,0.450250,0.442750,0.447500,0.424003,398820000,0.374557,-0.012867,0.006164,-0.007192,...,0.011453,1.046845,29.0,0.015704,1.169054,126.0,0.014002,1.232724,488.0,0.025212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-18,139.500000,141.550003,137.149994,140.149994,140.149994,221866000,11.405323,-0.037772,-0.012973,-0.025735,...,0.020210,1.087824,34.0,0.029339,1.432371,141.0,0.032505,3.320882,539.0,0.032855
2024-11-19,141.320007,147.130005,140.990005,147.009995,147.009995,227834900,11.422185,0.012962,0.047787,0.038664,...,0.030548,1.138045,35.0,0.029768,1.496544,142.0,0.032611,3.472427,539.0,0.032887
2024-11-20,147.410004,147.559998,142.729996,145.889999,145.889999,309871700,11.444923,0.042191,-0.007648,0.002918,...,0.030069,1.126817,34.0,0.029754,1.479289,142.0,0.032567,3.435222,538.0,0.032887


In [5]:
features = ['Log_Returns','Close_Ratio_2', 'Trend_2', 'Close_Ratio_5', 'Trend_5','Close_Ratio_60', 'Trend_60', 'Volatility_60',
       'Close_Ratio_250', 'Trend_250', 'Volatility_250', 'Close_Ratio_1000',
       'Trend_1000', 'Volatility_1000', 'log_Open', 'log_High', 'log_Low',
       'log_Close', 'log_Adj Close', 'log_Volume', 'log_VWAP']
target = ['Target']

In [6]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

# Assuming `data` is your DataFrame
features = features
target = ['Target']

# Split data into training (80%) and testing (20%) sets
split_index = int(len(data) * 0.8)
train_data = data.iloc[:split_index]
test_data = data.iloc[split_index:]

# Separate features (X) and target (y)
X_train = train_data[features]
y_train = train_data[target].values.ravel()
X_test = test_data[features]
y_test = test_data[target].values.ravel()

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Predict probabilities for the test set
y_prob = rf_model.predict_proba(X_test)[:, 1]  # Probability for the positive class

# Custom threshold for classification
custom_threshold = 0.4  # Change this to your desired threshold
y_pred = (y_prob >= custom_threshold).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print evaluation results
print(f"Accuracy: {accuracy:.2f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

# Optional: Display feature importances
feature_importances = pd.DataFrame({
    'Feature': features,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importances:")
print(feature_importances)


Accuracy: 0.47

Confusion Matrix:
[[180  58]
 [229  71]]

Classification Report:
              precision    recall  f1-score   support

           0       0.44      0.76      0.56       238
           1       0.55      0.24      0.33       300

    accuracy                           0.47       538
   macro avg       0.50      0.50      0.44       538
weighted avg       0.50      0.47      0.43       538


Feature Importances:
             Feature  Importance
19        log_Volume    0.069686
3      Close_Ratio_5    0.069337
5     Close_Ratio_60    0.062890
1      Close_Ratio_2    0.061768
0        Log_Returns    0.059998
7      Volatility_60    0.059362
8    Close_Ratio_250    0.055850
10    Volatility_250    0.055595
13   Volatility_1000    0.054274
11  Close_Ratio_1000    0.053242
20          log_VWAP    0.047804
15          log_High    0.045249
14          log_Open    0.044684
18     log_Adj Close    0.044031
16           log_Low    0.043499
17         log_Close    0.042459
12       

In [7]:
# Reset the index to make 'Date' a column
data = data.reset_index()

# Extract only the date part and convert to string format
data['Date'] = data['Date'].dt.date.astype(str)
data

,Date,Open,High,Low,Close,Adj Close,Volume,VWAP,log_diff_Open,log_diff_Close,...,Volatility_5,Close_Ratio_60,Trend_60,Volatility_60,Close_Ratio_250,Trend_250,Volatility_250,Close_Ratio_1000,Trend_1000,Volatility_1000
0,2014-03-24,0.463750,0.467500,0.454500,0.461250,0.437031,306096000,0.374334,-0.009657,-0.004866,...,0.012553,1.088292,31.0,0.015362,1.211940,127.0,0.014113,1.270916,489.0,0.025221
1,2014-03-25,0.466250,0.466750,0.457750,0.461250,0.437031,304424000,0.374377,0.005376,0.000000,...,0.008701,1.085326,31.0,0.015284,1.210096,126.0,0.014098,1.270831,488.0,0.025216
2,2014-03-26,0.465500,0.474250,0.446750,0.450750,0.427082,557756000,0.374447,-0.001610,-0.023027,...,0.009861,1.058264,30.0,0.015634,1.180938,125.0,0.014160,1.241818,488.0,0.025214
3,2014-03-27,0.449750,0.453500,0.439750,0.444750,0.421397,540728000,0.374509,-0.034420,-0.013400,...,0.009590,1.042322,29.0,0.015695,1.163579,125.0,0.014019,1.225234,487.0,0.025217
4,2014-03-28,0.444000,0.450250,0.442750,0.447500,0.424003,398820000,0.374557,-0.012867,0.006164,...,0.011453,1.046845,29.0,0.015704,1.169054,126.0,0.014002,1.232724,488.0,0.025212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2683,2024-11-18,139.500000,141.550003,137.149994,140.149994,140.149994,221866000,11.405323,-0.037772,-0.012973,...,0.020210,1.087824,34.0,0.029339,1.432371,141.0,0.032505,3.320882,539.0,0.032855
2684,2024-11-19,141.320007,147.130005,140.990005,147.009995,147.009995,227834900,11.422185,0.012962,0.047787,...,0.030548,1.138045,35.0,0.029768,1.496544,142.0,0.032611,3.472427,539.0,0.032887
2685,2024-11-20,147.410004,147.559998,142.729996,145.889999,145.889999,309871700,11.444923,0.042191,-0.007648,...,0.030069,1.126817,34.0,0.029754,1.479289,142.0,0.032567,3.435222,538.0,0.032887
2686,2024-11-21,149.350006,152.889999,140.699997,146.669998,146.669998,400946600,11.474503,0.013075,0.005332,...,0.030146,1.129779,35.0,0.029598,1.481257,143.0,0.032532,3.442784,539.0,0.032887


In [8]:
df2 = pd.read_csv("stock_news_sentiment_analysis_results_NVDA.csv")
df3 = df2.drop(['Ticker'],axis=1)
df = df3.drop(['weighted positive sentiment score', 'weighted negative sentiment score', 'weighted neutral sentiment score'],axis=1)
df

,Date,weighted compound sentiment score
0,2011-05-13,0.503300
1,2011-07-21,-0.225400
2,2011-07-25,-0.343000
3,2011-08-01,-0.709600
4,2011-08-11,0.654800
...,...,...
1481,2024-11-05,0.218962
1482,2024-11-06,0.603694
1483,2024-11-07,0.540471
1484,2024-11-08,0.345862


In [9]:
# Merge df and data on the 'Date' column
# This performs a left join, keeping all rows from df and adding matching rows from data
df = pd.merge(df, data, on='Date', how='left')

# Display the modified df
df

,Date,weighted compound sentiment score,Open,High,Low,Close,Adj Close,Volume,VWAP,log_diff_Open,...,Volatility_5,Close_Ratio_60,Trend_60,Volatility_60,Close_Ratio_250,Trend_250,Volatility_250,Close_Ratio_1000,Trend_1000,Volatility_1000
0,2011-05-13,0.503300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-07-21,-0.225400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-07-25,-0.343000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-08-01,-0.709600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-08-11,0.654800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1481,2024-11-05,0.218962,137.449997,140.369995,137.330002,139.910004,139.910004,160537400.0,11.268754,0.001748,...,0.030416,1.113372,37.0,0.031180,1.482816,144.0,0.032343,3.411496,540.0,0.032819
1482,2024-11-06,0.603694,142.960007,146.490005,141.960007,145.610001,145.610001,242043900.0,11.286519,0.039305,...,0.034426,1.154220,37.0,0.030570,1.536774,144.0,0.032416,3.539064,541.0,0.032837
1483,2024-11-07,0.540471,146.389999,148.929993,146.169998,148.880005,148.880005,207323300.0,11.302102,0.023709,...,0.012793,1.175358,37.0,0.030619,1.564554,144.0,0.032434,3.606676,541.0,0.032840
1484,2024-11-08,0.345862,148.770004,149.770004,146.259995,147.630005,147.630005,175665800.0,11.315183,0.016127,...,0.019163,1.161703,36.0,0.030293,1.544969,143.0,0.032407,3.564804,541.0,0.032841


In [10]:
df = df.dropna()

In [11]:
df

,Date,weighted compound sentiment score,Open,High,Low,Close,Adj Close,Volume,VWAP,log_diff_Open,...,Volatility_5,Close_Ratio_60,Trend_60,Volatility_60,Close_Ratio_250,Trend_250,Volatility_250,Close_Ratio_1000,Trend_1000,Volatility_1000
38,2014-04-01,0.000000,0.462000,0.471750,0.456500,0.466250,0.441768,656272000.0,0.374688,0.030214,...,0.024311,1.085386,31.0,0.016284,1.214174,127.0,0.014181,1.284173,488.0,0.025238
39,2014-05-06,-0.659700,0.470250,0.474250,0.455000,0.456250,0.432293,416188000.0,0.375748,0.021495,...,0.012790,0.997032,33.0,0.016219,1.149140,126.0,0.014208,1.250929,492.0,0.024724
40,2014-05-08,-0.386400,0.455000,0.469750,0.452750,0.462500,0.438215,432728000.0,0.375847,-0.002743,...,0.013723,1.006474,33.0,0.016100,1.162446,126.0,0.013960,1.267090,493.0,0.024703
41,2014-05-19,0.433100,0.453750,0.466000,0.451750,0.463500,0.439163,360076000.0,0.376134,0.016107,...,0.018760,1.006870,30.0,0.015308,1.157725,124.0,0.014193,1.266511,490.0,0.024638
42,2014-08-07,0.239400,0.444750,0.448750,0.433500,0.436500,0.415485,407696000.0,0.378189,0.014153,...,0.007541,0.937559,30.0,0.012748,1.028933,128.0,0.014260,1.157317,493.0,0.023559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1480,2024-11-04,-0.052257,137.210007,138.960007,135.570007,136.050003,136.050003,187528200.0,11.257469,0.018463,...,0.026239,1.087109,37.0,0.031373,1.447672,144.0,0.032308,3.327636,540.0,0.032809
1481,2024-11-05,0.218962,137.449997,140.369995,137.330002,139.910004,139.910004,160537400.0,11.268754,0.001748,...,0.030416,1.113372,37.0,0.031180,1.482816,144.0,0.032343,3.411496,540.0,0.032819
1482,2024-11-06,0.603694,142.960007,146.490005,141.960007,145.610001,145.610001,242043900.0,11.286519,0.039305,...,0.034426,1.154220,37.0,0.030570,1.536774,144.0,0.032416,3.539064,541.0,0.032837
1483,2024-11-07,0.540471,146.389999,148.929993,146.169998,148.880005,148.880005,207323300.0,11.302102,0.023709,...,0.012793,1.175358,37.0,0.030619,1.564554,144.0,0.032434,3.606676,541.0,0.032840


In [12]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

# Assuming `df` is your DataFrame
features = ['weighted compound sentiment score','Log_Returns','Close_Ratio_2', 'Trend_2', 'Close_Ratio_5', 'Trend_5','Close_Ratio_60', 'Trend_60', 'Volatility_60',
       'Close_Ratio_250', 'Trend_250', 'Volatility_250', 'Close_Ratio_1000',
       'Trend_1000', 'Volatility_1000', 'log_Open', 'log_High', 'log_Low',
       'log_Close', 'log_Adj Close', 'log_Volume', 'log_VWAP']
target = ['Target']
# Split data into training (80%) and testing (20%) sets
split_index = int(len(df) * 0.8)
train_data = df.iloc[:split_index]
test_data = df.iloc[split_index:]

# Separate features (X) and target (y)
X_train = train_data[features]
y_train = train_data[target]
X_test = test_data[features]
y_test = test_data[target]

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Predict probabilities for the test set
y_prob = rf_model.predict_proba(X_test)[:, 1]  # Probability for the positive class

# Custom threshold for classification
custom_threshold = 0.45  # Change this to your desired threshold
y_pred = (y_prob >= custom_threshold).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print evaluation results
print(f"Accuracy: {accuracy:.2f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

# Optional: Display feature importances
feature_importances = pd.DataFrame({
    'Feature': features,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importances:")
print(feature_importances)


C:\Users\yaman\AppData\Local\Temp\ipykernel_11860\836133891.py:25: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(X_train, y_train)


Accuracy: 0.49

Confusion Matrix:
[[76 25]
 [98 40]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.44      0.75      0.55       101
         1.0       0.62      0.29      0.39       138

    accuracy                           0.49       239
   macro avg       0.53      0.52      0.47       239
weighted avg       0.54      0.49      0.46       239


Feature Importances:
                              Feature  Importance
2                       Close_Ratio_2    0.063027
1                         Log_Returns    0.062725
20                         log_Volume    0.062270
4                       Close_Ratio_5    0.061632
8                       Volatility_60    0.058304
6                      Close_Ratio_60    0.056090
14                    Volatility_1000    0.055909
0   weighted compound sentiment score    0.055533
11                     Volatility_250    0.052275
9                     Close_Ratio_250    0.050330
12                   Clo

In [13]:
y_test

,Target
1167,0.0
1168,0.0
1169,0.0
1170,0.0
1171,1.0
...,...
1480,1.0
1481,1.0
1482,1.0
1483,0.0


In [20]:
features = ['Log_Returns','Close_Ratio_2', 'Trend_2', 'Close_Ratio_5', 'Trend_5','Close_Ratio_60', 'Trend_60', 'Volatility_60',
       'Close_Ratio_250', 'Trend_250', 'Volatility_250', 'Close_Ratio_1000',
       'Trend_1000', 'Volatility_1000', 'log_Open', 'log_High', 'log_Low',
       'log_Close', 'log_Adj Close', 'log_Volume', 'log_VWAP']
target = ['Target']


import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

# Assuming `df` is your DataFrame
features = features
target = 'Target'

# Split data into training (80%) and testing (20%) sets
split_index = int(len(df) * 0.8)
train_data = df.iloc[:split_index]
test_data = df.iloc[split_index:]

# Separate features (X) and target (y)
X_train = train_data[features]
y_train = train_data[target]
X_test = test_data[features]
y_test = test_data[target]

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Predict probabilities for the test set
y_prob = rf_model.predict_proba(X_test)[:, 1]  # Probability for the positive class

# Custom threshold for classification
custom_threshold = 0.45  # Change this to your desired threshold
y_pred = (y_prob >= custom_threshold).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print evaluation results
print(f"Accuracy: {accuracy:.2f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

# Optional: Display feature importances
feature_importances = pd.DataFrame({
    'Feature': features,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importances:")
print(feature_importances)


Accuracy: 0.53

Confusion Matrix:
[[57 44]
 [69 69]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.45      0.56      0.50       101
         1.0       0.61      0.50      0.55       138

    accuracy                           0.53       239
   macro avg       0.53      0.53      0.53       239
weighted avg       0.54      0.53      0.53       239


Feature Importances:
             Feature  Importance
1      Close_Ratio_2    0.068561
3      Close_Ratio_5    0.067031
19        log_Volume    0.066953
0        Log_Returns    0.066133
5     Close_Ratio_60    0.064670
7      Volatility_60    0.058750
13   Volatility_1000    0.057541
11  Close_Ratio_1000    0.055989
8    Close_Ratio_250    0.054457
10    Volatility_250    0.053055
20          log_VWAP    0.050046
14          log_Open    0.043597
16           log_Low    0.041890
15          log_High    0.041819
17         log_Close    0.039336
18     log_Adj Close    0.037789
6           T